In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from numpy import isnan
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score
import random
!pip install --upgrade category_encoders
import category_encoders as ce
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 86 kB 3.0 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [11]:
data = '/content/winequality-red.csv'

df = pd.read_csv(data, sep=',')
df.shape
df.head(5)

df['quality'] = df['quality'].astype('str')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   object 
dtypes: float64(11), object(1)
memory usage: 150.0+ KB


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


# Gaussian approach + null score

In [21]:
#Train-test split
X = df.drop(['quality'], axis=1)
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

#Scaling
cols = X_train.columns

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])

#Fitting the model
gnb = GaussianNB()
gnb.fit(X_train, y_train)

#Predictions
y_pred = gnb.predict(X_test)
y_pred

#Accuracy
print('Gaussian model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

y_test.value_counts() # 5 -> 208 // others -> 204 + 45 + 15 + 5 + 3 
null_accuracy = (208/(208+272))
print('Null accuracy score: {0:0.4f}'. format(null_accuracy))


Gaussian model accuracy score: 0.5521
Null accuracy score: 0.4333


# Discretization approach (EWD, EFD, PD, FFD and quantiles)

In [30]:
#!pip install feature_engine
#!pip install -U feature-engine
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine import discretisation as dsc

# Varying the number of bins (q) of the EqualFrequencyDiscretizer we can generate the different discretization values. More specifically:
# Quantiles -> q=4 (we divide by quantiles, i.e. four equal frequency bins)
# EFD -> q=10 (standard value for EFD)
# FFD -> q=1628 (total number of instances divided by 30, which is the recommended size of the bins for FFD)
# PD -> q=221 (square root of the number of instances)
# In order to execute the EWD, the function must be uncommented. The standard value are 10 bins
discretizer = dsc.EqualFrequencyDiscretiser(q=54) 
#discretizer = EqualWidthDiscretiser(bins=10)
discretizer.fit(df)
df_efd = discretizer.fit_transform(df)

cols=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

for column in cols:
  print(column)
  df_efd[column] = df_efd[column].astype('object')

for var in df_efd.columns: 
    print(df_efd[var].value_counts())
    print('\n')

df_efd.info()

fixed acidity
volatile acidity
citric acid
residual sugar
chlorides
free sulfur dioxide
total sulfur dioxide
density
pH
sulphates
alcohol
11    67
26    58
10    57
17    53
14    52
9     50
33    50
16    49
7     46
15    46
21    45
12    44
13    44
0     43
4     42
18    42
19    42
22    40
8     38
30    38
5     37
2     36
27    33
36    32
39    32
31    31
40    30
43    30
42    29
29    29
35    29
1     28
6     28
25    27
23    26
28    26
20    26
37    26
41    25
34    23
38    20
24    19
32    17
3     14
Name: fixed acidity, dtype: int64


21    61
4     49
33    48
22    46
16    43
1     41
30    41
13    39
32    39
9     38
2     37
38    36
12    35
11    35
28    34
15    34
40    33
14    33
24    33
49    32
0     32
37    32
26    31
19    31
48    31
44    31
46    30
43    30
36    30
35    30
7     30
25    29
29    29
50    28
39    28
51    28
45    28
41    26
42    26
10    26
5     25
47    25
27    25
23    24
17    23
8     22
18    22
20    2

In [31]:
#Train-test split
X = df_efd.drop(['quality'], axis=1)
y = df_efd['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

#One-hot encoding
encoder = ce.OneHotEncoder(cols=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'])
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

#Fitting the model
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

#Predictions
y_pred = mnb.predict(X_test)
y_pred

#Accuracy
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score: 0.5604
